$ \frac{1}{4} \sum_{m<n} J_{HS}(m-n) (\sigma^x_{m} \sigma^x_{n} + \sigma^y_{m} \sigma^y_{n} + \sigma^z_{m} \sigma^z_n) $ where $J_{HS}(m-n) = \frac{\pi^2}{N^2 \sin^2 ((m-n)\pi / N)}$, $N$ is number of sites. Use periodic boundary conditions

In [1]:
import numpy as np
import netket as nk
import flax.linen  as nn
from netket.operator.spin import sigmax, sigmay, sigmaz

In [2]:
def wrap_index(n, i, j=0):
    wrap = (abs(i - j) - 1) % n + 1
    if j != 0:
        if wrap > n / 2:
            return abs(n - abs(wrap))
    return wrap

In [6]:
hi = nk.hilbert.Spin(s=1/2, N=20)
def haldane_shastry_model(N):
    sum = 0
    for n in range(N):
        for m in range(n):
            hs = (np.pi**2) / (N**2 * (np.sin((wrap_index(n, m))*np.pi/N)**2))
            x = sigmax(hi, m) * sigmax(hi, n)
            y = sigmay(hi, m) * sigmay(hi, n)
            z = sigmaz(hi, m) * sigmaz(hi, n)
            sum += hs * (x+y+z)
    return sum / 4



In [7]:
hs20 = haldane_shastry_model(20)
print(hs20)

LocalOperator(dim=20, #acting_on=190 locations, constant=0j, dtype=complex128)


In [8]:
rbm = nk.models.RBM(alpha=1, param_dtype=float, kernel_init=nn.initializers.normal(stddev=0.01))
sampler = nk.sampler.MetropolisLocal(hi)
vstate = nk.vqs.MCState(sampler, model=rbm, n_samples=500)

optimizer = nk.optimizer.Sgd(learning_rate=0.1)
preconditioner = nk.optimizer.SR(diag_shift=0.1)
vmc = nk.driver.VMC(hs20, optimizer, variational_state=vstate, preconditioner=preconditioner)

log = nk.logging.RuntimeLog()
vmc.run(n_iter=300, out=log)
opt_energy = vstate.expect(hs20)
print(opt_energy)

/Users/adamremels/miniconda3/envs/csci1470/lib/python3.10/site-packages/netket/vqs/mc/mc_state/state.py:257: UserWarning: n_samples=500 (500 per device/MPI rank) does not divide n_chains=16, increased to 512 (512 per device/MPI rank)
  self.n_samples = n_samples
100%|██████████| 300/300 [00:07<00:00, 37.67it/s, Energy=-3.8333+0.0000j ± 0.0011 [σ²=0.0006, R̂=1.0146]]    


-3.834e+00+0.000e+00j ± nan [σ²=0.000e+00]
